# Category Boosting (CatBoost) (Yandex 2017) (Kategorik değişkenler için iyi)
* Kategorik değişkenler ile otomatik olarak mücadelee edebilen hızlı başarılı bir diğer GBM türevi.
* Çok fazla kategorik değişken olduğunda etkilidir. bazen bir columnu one hot encodin yöntemi ile ayırdığımız zamanlarda, çok fazla kategorik değişken oluşmuş oluyor.
* Kategorik değişken desteği.
* Hızlı ve ölçeklenebilir GPU desteği
* Daha başarılı tahminler
* Hızlı train ve hızlı tahmin
* Rusyanın ilk açık kaynak kodlu, başarılı ML çalışması. !

libraray

In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler ## standartlaStırma
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [33]:
df = pd.read_csv('Hitters.csv')
df = df.dropna()
y= df['Salary']
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']]) ## one hot encoding
X_= df.drop(['Salary', 'League', 'Division','NewLeague'], axis =1).astype('float64')
X= pd.concat([X_,dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)

model ve tahmin

In [34]:
!pip install catboost

  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/e2/63/379617e3d982e8a66c9d66ebf4621d3357c7c18ad356473c335bffd5aba6/catboost-1.2.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/47.0 kB ? eta -:--:--
     -------- ------------------------------- 10.2/47.0 kB ? eta -:--:--
     -------- ------------------------------- 10.2/47.0 kB ? eta -:--

In [36]:
from catboost import CatBoostRegressor

In [38]:
catb = CatBoostRegressor().fit(X_train,y_train)

Learning rate set to 0.031674
0:	learn: 437.6430699	total: 138ms	remaining: 2m 18s
1:	learn: 431.3923642	total: 140ms	remaining: 1m 9s
2:	learn: 424.8820360	total: 142ms	remaining: 47.1s
3:	learn: 418.2514904	total: 143ms	remaining: 35.6s
4:	learn: 412.6394021	total: 145ms	remaining: 28.8s
5:	learn: 406.6247020	total: 146ms	remaining: 24.2s
6:	learn: 400.5321206	total: 148ms	remaining: 21s
7:	learn: 394.6683437	total: 150ms	remaining: 18.5s
8:	learn: 388.2496484	total: 151ms	remaining: 16.7s
9:	learn: 382.9448842	total: 153ms	remaining: 15.1s
10:	learn: 377.2600080	total: 155ms	remaining: 13.9s
11:	learn: 372.4829606	total: 156ms	remaining: 12.9s
12:	learn: 366.6823437	total: 158ms	remaining: 12s
13:	learn: 362.6076230	total: 159ms	remaining: 11.2s
14:	learn: 358.0107745	total: 161ms	remaining: 10.6s
15:	learn: 353.2802665	total: 163ms	remaining: 10s
16:	learn: 348.5646265	total: 164ms	remaining: 9.49s
17:	learn: 343.6407912	total: 166ms	remaining: 9.05s
18:	learn: 339.2363847	total: 1

In [40]:
y_pred = catb.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

351.194631344607

Tuning

In [44]:
?catb

Type:           CatBoostRegressor
String form:    <catboost.core.CatBoostRegressor object at 0x0000018D0ACFF910>
File:           c:\users\ycanf\anaconda3\lib\site-packages\catboost\core.py
Docstring:     
Implementation of the scikit-learn API for CatBoost regression.

Parameters
----------
Like in CatBoostClassifier, except loss_function, classes_count, class_names and class_weights

loss_function : string, [default='RMSE']
    'RMSE'
    'MAE'
    'Quantile:alpha=value'
    'LogLinQuantile:alpha=value'
    'Poisson'
    'MAPE'
    'Lq:q=value'
    'SurvivalAft:dist=value;scale=value'
Init docstring:
Initialize the CatBoost.

Parameters
----------
params : dict
    Parameters for CatBoost.
    If  None, all params are set to their defaults.
    If  dict, overriding parameters present in dict.

In [45]:
catb_params = {'iterations' : [200,500,1000],
               'learning_rate' : [0.01,0.1],
               'depth' : [3,6,8]}

In [46]:
catb = CatBoostRegressor()

In [47]:
catb_cv= GridSearchCV(catb,catb_params,
                      cv=5,n_jobs=-1,verbose=2).fit(X_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
0:	learn: 425.7900818	total: 723us	remaining: 144ms
1:	learn: 404.8723520	total: 1.67ms	remaining: 166ms
2:	learn: 387.4057666	total: 2.39ms	remaining: 157ms
3:	learn: 372.2801584	total: 3.1ms	remaining: 152ms
4:	learn: 358.9204229	total: 3.8ms	remaining: 148ms
5:	learn: 347.0083933	total: 4.41ms	remaining: 143ms
6:	learn: 336.0130818	total: 5.08ms	remaining: 140ms
7:	learn: 324.3923300	total: 5.64ms	remaining: 135ms
8:	learn: 314.8690957	total: 6.2ms	remaining: 132ms
9:	learn: 308.5075563	total: 6.84ms	remaining: 130ms
10:	learn: 298.8587285	total: 7.4ms	remaining: 127ms
11:	learn: 294.7655438	total: 8.03ms	remaining: 126ms
12:	learn: 288.0697862	total: 8.69ms	remaining: 125ms
13:	learn: 282.6697154	total: 9.36ms	remaining: 124ms
14:	learn: 277.6121667	total: 10.2ms	remaining: 126ms
15:	learn: 273.4383979	total: 10.7ms	remaining: 124ms
16:	learn: 269.1556201	total: 11.3ms	remaining: 122ms
17:	learn: 264.8098704	total: 12ms	r

In [48]:
catb_cv.best_params_

{'depth': 3, 'iterations': 200, 'learning_rate': 0.1}

In [49]:
catb_tuned= CatBoostRegressor(depth=3, iterations = 200, learning_rate = 0.1).fit(X_train,y_train)

0:	learn: 425.7900818	total: 895us	remaining: 178ms
1:	learn: 404.8723520	total: 1.77ms	remaining: 175ms
2:	learn: 387.4057666	total: 2.56ms	remaining: 168ms
3:	learn: 372.2801584	total: 3.23ms	remaining: 158ms
4:	learn: 358.9204229	total: 3.91ms	remaining: 152ms
5:	learn: 347.0083933	total: 4.51ms	remaining: 146ms
6:	learn: 336.0130818	total: 5.24ms	remaining: 144ms
7:	learn: 324.3923300	total: 5.91ms	remaining: 142ms
8:	learn: 314.8690957	total: 6.5ms	remaining: 138ms
9:	learn: 308.5075563	total: 7.11ms	remaining: 135ms
10:	learn: 298.8587285	total: 7.87ms	remaining: 135ms
11:	learn: 294.7655438	total: 8.48ms	remaining: 133ms
12:	learn: 288.0697862	total: 9.21ms	remaining: 132ms
13:	learn: 282.6697154	total: 10.1ms	remaining: 134ms
14:	learn: 277.6121667	total: 10.8ms	remaining: 134ms
15:	learn: 273.4383979	total: 11.5ms	remaining: 133ms
16:	learn: 269.1556201	total: 12.4ms	remaining: 134ms
17:	learn: 264.8098704	total: 13.1ms	remaining: 133ms
18:	learn: 261.6700768	total: 14ms	remai

In [50]:
y_pred=catb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

344.3125832615482